### Multilayered Feedforward and Backpropagation Example

This is a borrowed example from a lession on "Implementing Gradient Descent". I've just structured and zipped comments, math and code in one flow.

In [6]:
import numpy as np


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


x = np.array([.1, .3])
target = 1
learnrate = 0.5

weights_input_hidden = np.array([0.4, -0.2])

weights_hidden_output = np.array([0.1])

<img src="https://d17h27t6h515a5.cloudfront.net/topher/2017/January/588bb45d_backprop-network/backprop-network.png" style="width:100px;height:200px">

Assume we're trying to fit some binary data and the target is $y = 1$. We'll start with the forward pass, first calculating the input to the hidden unit

$h = \sum_i w_i x_i = 0.1 \times 0.4 - 0.2 \times 0.3 = -0.02$

In [7]:
hidden_layer_input = np.dot(x, weights_input_hidden)

and the output of the hidden unit

$a = f(h) = sigmoid(-0.02) = 0.495$

In [8]:
hidden_layer_output = sigmoid(hidden_layer_input)

Using this as the input to the output unit, the output of the network is

$\hat{y} = f( W \cdot a ) = sigmoid( 0.1 \times 0.495 ) = 0.512$

In [9]:
output_layer_in = np.dot(hidden_layer_output, weights_hidden_output)
output = sigmoid(output_layer_in)

With the network output, we can start the backwards pass to calculate the weight updates for both layers. Using the fact that for the sigmoid function $f'(W \cdot a) = f(W \cdot a) (1 - f(W \cdot a))$, the error term for the output unit is

$\delta^0 = (y - \hat{y})f'(W \cdot a) = (1 - 0.512) \times 0.512 \times (1 - 0.512) = 0.122$

In [11]:
error = target - output
output_error_term = error * output * (1 - output)

Now we need to calculate the error term for the hidden unit with backpropagation. Here we'll scale the error term from the output unit by the weight $W$ connecting it to the hidden unit. For the hidden unit error term, $\delta^h_j = \sum_k W_{jk} \delta^o_k f'(h_j)$, but since we have one hidden unit and one output unit, this is much simpler.

$\delta^h = W \delta^0 f'(h) = 0.1 \times 0.122 \times 0.495 \times (1 - 0.495) = 0.003$

In [13]:
hidden_error_term = np.dot(output_error_term,weights_hidden_output) * \
                    hidden_layer_output * (1 - hidden_layer_output)

Now that we have the errors, we can calculate the gradient descent steps. The hidden to output weight step is the learning rate, times the output unit error, times the hidden unit activation value.

$\Delta W = \eta\delta^0a = 0.5 \times 0.122 \times 0.495 = 0.0302$

In [15]:
delta_w_h_o = learnrate * output_error_term * hidden_layer_output

Then, for the input to hidden weights $w_i$, it's the learning rate times the hidden unit error, times the input values.

$\Delta w_i = \eta\delta^h x_i = (0.5 \times 0.003 \times 0.1,0.5 \times 0.003 \times 0.3) = (0.00015,0.00045)$

In [16]:
delta_w_i_h = learnrate * hidden_error_term * x[:,None]

From this example, you can see one of the effects of using the sigmoid function for the activations. The maximum derivative of the sigmoid function is 0.25, so the errors in the output layer get reduced by at least 75%, and errors in the hidden layer are scaled down by at least 93.75%! You can see that if you have a lot of layers, using a sigmoid activation function will quickly reduce the weight steps to tiny values in layers near the input. This is known as the vanishing gradient problem. Later in the course you'll learn about other activation functions that perform better in this regard and are more commonly used in modern network architectures.

In [17]:
print('Change in weights for hidden layer to output layer:')
print(delta_w_h_o)
print('Change in weights for input layer to hidden layer:')
print(delta_w_i_h)

Change in weights for hidden layer to output layer:
[ 0.03015349]
Change in weights for input layer to hidden layer:
[[ 0.00015228]
 [ 0.00045683]]
